**Import library yang digunakan**

In [1]:
#import library yang akan digunakan
import pandas as pd
import seaborn as sns
import os
%matplotlib inline
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report
import numpy as np
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

#ambil API KAGGLE
os.environ['KAGGLE_USERNAME'] = "johanesandre"
os.environ['KAGGLE_KEY'] = "a54c201c45be4ec682e929488efb0ba3"

**Download Dataset dari Kaggle**

In [2]:
#Download Dataset Kaggle
!kaggle datasets download -d swarnimrai/webtoon-comics-dataset/

  0% 0.00/106k [00:00<?, ?B/s]
100% 106k/106k [00:00<00:00, 40.3MB/s]


**Persiapan Import Data ke Lembar Kerja**

In [3]:
# melakukan ekstraksi pada file zip
import zipfile,os
local_zip = '/content/webtoon-comics-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

**Persiapan Import Data ke Lembar Kerja**

In [4]:
#membaca Data CSV dan melihat 5 data teratas
df = pd.read_csv('/tmp/Webtoon Dataset.csv')
df.head()

,id,Name,Writer,Likes,Genre,Rating,Subscribers,Summary,Update,Reading Link
0,0,Let's Play,Leeanne M. Krecic (Mongie),30.6M,Romance,9.62,4.2M,"She's young, single and about to achieve her d...",UP EVERY TUESDAY,https://www.webtoons.com/en/romance/letsplay/l...
1,1,True Beauty,Yaongyi,39.9M,Romance,9.60,6.4M,"After binge-watching beauty videos online, a s...",UP EVERY WEDNESDAY,https://www.webtoons.com/en/romance/truebeauty...
2,2,Midnight Poppy Land,Lilydusk,10.4M,Romance,9.81,2.1M,After making a grisly discovery in the country...,UP EVERY SATURDAY,https://www.webtoons.com/en/romance/midnight-p...
3,3,Age Matters,Enjelicious,25.9M,Romance,9.79,3.5M,She's a hopeless romantic who's turning 30's ...,UP EVERY WEDNESDAY,https://www.webtoons.com/en/romance/age-matter...
4,4,Unholy Blood,Lina Im / Jeonghyeon Kim,9.9M,Supernatural,9.85,1.5M,When vampires destroy her chance to have the n...,UP EVERY THURSDAY,https://www.webtoons.com/en/supernatural/unhol...


In [5]:
# Mengecek total baris dan kolom dari dataset
df.shape


(569, 10)

**Buat Fungsi untuk menghilangkan tanda baca di kolom summary. Tujuannya untuk mendapatkan pendekatan judul yang lebih baik**

In [6]:
#buat fungsi untuk menghilangkan tanda baca pada kolom summary
import string
def remove_punctuation(text):
    punctuationfree = "".join([i for i in text if i not in string.punctuation])
    return punctuationfree

**Terapkan fungsi remove_punctuation untuk kolom summary, simpan hasil baru pada kolom clean_summary**

In [7]:
#Bersihkan target kolom clean summary
df['Clean_Summary'] = df['Summary'].apply(lambda x:remove_punctuation(x))
df.head()

,id,Name,Writer,Likes,Genre,Rating,Subscribers,Summary,Update,Reading Link,Clean_Summary
0,0,Let's Play,Leeanne M. Krecic (Mongie),30.6M,Romance,9.62,4.2M,"She's young, single and about to achieve her d...",UP EVERY TUESDAY,https://www.webtoons.com/en/romance/letsplay/l...,Shes young single and about to achieve her dre...
1,1,True Beauty,Yaongyi,39.9M,Romance,9.60,6.4M,"After binge-watching beauty videos online, a s...",UP EVERY WEDNESDAY,https://www.webtoons.com/en/romance/truebeauty...,After bingewatching beauty videos online a shy...
2,2,Midnight Poppy Land,Lilydusk,10.4M,Romance,9.81,2.1M,After making a grisly discovery in the country...,UP EVERY SATURDAY,https://www.webtoons.com/en/romance/midnight-p...,After making a grisly discovery in the country...
3,3,Age Matters,Enjelicious,25.9M,Romance,9.79,3.5M,She's a hopeless romantic who's turning 30's ...,UP EVERY WEDNESDAY,https://www.webtoons.com/en/romance/age-matter...,Shes a hopeless romantic whos turning 30s and...
4,4,Unholy Blood,Lina Im / Jeonghyeon Kim,9.9M,Supernatural,9.85,1.5M,When vampires destroy her chance to have the n...,UP EVERY THURSDAY,https://www.webtoons.com/en/supernatural/unhol...,When vampires destroy her chance to have the n...


**Cek Deskripsi Tiap Kolomnya**

In [8]:
# Memuat deskripsi setiap kolom dataframe
df.describe

<bound method NDFrame.describe of       id  ...                                      Clean_Summary
0      0  ...  Shes young single and about to achieve her dre...
1      1  ...  After bingewatching beauty videos online a shy...
2      2  ...  After making a grisly discovery in the country...
3      3  ...  Shes a hopeless romantic whos turning 30s  and...
4      4  ...  When vampires destroy her chance to have the n...
..   ...  ...                                                ...
564  564  ...  Lifes funny One minute youre jobless deep in d...
565  565  ...  Adamsville is an all ages series about two mid...
566  566  ...                         Get your WEBTOON news here
567  567  ...  A series of tips for staying safe during the C...
568  568  ...  The mysterious team that brings Webtoon to you...

[569 rows x 11 columns]>

In [9]:
# Memuat informasi dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             569 non-null    int64  
 1   Name           569 non-null    object 
 2   Writer         568 non-null    object 
 3   Likes          569 non-null    object 
 4   Genre          569 non-null    object 
 5   Rating         569 non-null    float64
 6   Subscribers    569 non-null    object 
 7   Summary        569 non-null    object 
 8   Update         569 non-null    object 
 9   Reading Link   569 non-null    object 
 10  Clean_Summary  569 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 49.0+ KB


***Persiapan Tahap Data Processing, Recheck kembali kolom yang akan dikelola. Tampilkan 5 data teratas***

In [10]:
#Tampilkan data 5 teratas di setiap variabel
columns = ["Name", "Writer", "Genre", "Update", "Clean_Summary"]
df[columns].head()

,Name,Writer,Genre,Update,Clean_Summary
0,Let's Play,Leeanne M. Krecic (Mongie),Romance,UP EVERY TUESDAY,Shes young single and about to achieve her dre...
1,True Beauty,Yaongyi,Romance,UP EVERY WEDNESDAY,After bingewatching beauty videos online a shy...
2,Midnight Poppy Land,Lilydusk,Romance,UP EVERY SATURDAY,After making a grisly discovery in the country...
3,Age Matters,Enjelicious,Romance,UP EVERY WEDNESDAY,Shes a hopeless romantic whos turning 30s and...
4,Unholy Blood,Lina Im / Jeonghyeon Kim,Supernatural,UP EVERY THURSDAY,When vampires destroy her chance to have the n...


***Cek Data set yang digunakan, terutama pada kolom clean_summary***

In [11]:
#Cek apa semua data ada yang kosong tidak
df[columns].isnull().sum()

Name             0
Writer           1
Genre            0
Update           0
Clean_Summary    0
dtype: int64

**Buat fungsi similarity dan persiapkan variabel similarity untuk diisi, kolom ini nanti akan digunakan untuk proses Countvector**

In [12]:
#buat fungsi ambil data yang mirip. Data yang ditampung merupakan Nama, penulis, Genre, dan update
def get_similar(data):
  similar = []
  for i in range(0, data.shape[0]):
    similar.append(str(data['Name'][i]) + ' '+ str(data['Writer'][i]) + ' '+ str(data['Genre'][i]) + ' '+ str(data['Update'][i]))

  return similar

In [13]:
#tampilkan data dengan fungsi similarity yang dibuat tadi
df['similar'] = get_similar(df)
df.head(3)

,id,Name,Writer,Likes,Genre,Rating,Subscribers,Summary,Update,Reading Link,Clean_Summary,similar
0,0,Let's Play,Leeanne M. Krecic (Mongie),30.6M,Romance,9.62,4.2M,"She's young, single and about to achieve her d...",UP EVERY TUESDAY,https://www.webtoons.com/en/romance/letsplay/l...,Shes young single and about to achieve her dre...,Let's Play Leeanne M. Krecic (Mongie) Romance ...
1,1,True Beauty,Yaongyi,39.9M,Romance,9.60,6.4M,"After binge-watching beauty videos online, a s...",UP EVERY WEDNESDAY,https://www.webtoons.com/en/romance/truebeauty...,After bingewatching beauty videos online a shy...,True Beauty Yaongyi Romance UP EVERY WEDNESDAY
2,2,Midnight Poppy Land,Lilydusk,10.4M,Romance,9.81,2.1M,After making a grisly discovery in the country...,UP EVERY SATURDAY,https://www.webtoons.com/en/romance/midnight-p...,After making a grisly discovery in the country...,Midnight Poppy Land Lilydusk Romance UP EVERY ...


**Pada Kolom similar yang didapat tadi, lakukan fungsi CountVectorizer dengan lib sklearn. Fungsi ini digunakan mengonversi dokument teks menjadi matriks dalam jumlah token**

In [14]:
#ubah kolom similar column menjadi tipe datavector
cm = CountVectorizer().fit_transform(df['similar'])

**Lakukan Cosine Similarity Dengan Lib sklearn, Proses ini digunakan untuk menghitung kesamaan antara dua buah objek yang dinyatakan dalam dua buah vector**

In [15]:
#Dapatkan nilai similarity/kemiripan matrix dengan fungsi Cosine similarity

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

cs = cosine_similarity(cm)
print(cs)

[[1.         0.37796447 0.35355339 ... 0.21081851 0.         0.        ]
 [0.37796447 1.         0.40089186 ... 0.23904572 0.         0.        ]
 [0.35355339 0.40089186 1.         ... 0.2236068  0.         0.        ]
 ...
 [0.21081851 0.23904572 0.2236068  ... 1.         0.11952286 0.31622777]
 [0.         0.         0.         ... 0.11952286 1.         0.37796447]
 [0.         0.         0.         ... 0.31622777 0.37796447 1.        ]]


**Cek Total Baris & Kolom hasil cosine similarity**

In [16]:
# Mengecek total baris dan kolom dari hasil cosine similarity
cs.shape

(569, 569)

**Lakukan Uji Coba Pada Model yang telah dibuat. Ambil satu judul yang ada di dalam dataset**

**Pada uji Coba ini kita melakukan 2 Judul Webtoon. Variable title untuk Testcase 1 dan title2 untuk testcast 2**

In [17]:
#Uji coba dengan salah satu judul yang ada di dataset
title = 'About Death'
title2 = 'Delusion'
comic_id = df[df.Name == title]['id'].values[0]
comic_id2= df[df.Name == title2]['id'].values[0]

**Lakukan Enumerate dengan fungsi bawaan lib python.  Tujuannya untuk mengembalikan objek iterable (indeks dan itemnya) bersamaan.**

In [18]:
#Cari enumerasi matrix kesamaan pada judul
scores = list(enumerate(cs[comic_id]))
scores2 = list(enumerate(cs[comic_id2]))
#print (scores)

**Dari hasil yang didapatkan, skor di sortir berdasarkan Descending dengan fungsi bawaan python. Angka tertinggi menunjukan kemiripan dengan data test yang dimaksud**

In [19]:
#Sortir rekomendasi dari yang paling tinggi ke paling rendah
sorted_scores = sorted(scores, key=lambda x:x[1], reverse=True)
sorted_scores = sorted_scores[1:]

sorted_scores2 = sorted(scores2, key=lambda x:x[1], reverse=True)
sorted_scores2 = sorted_scores2[1:]
#print(sorted_scores[:10])

**Tampilkan Hasil Rekomendasi yang didapatkan. Hasil Rekomendasi merupakan Top 10 Data yang skor paling tinggi**

In [20]:
#Tampilkan hasil rekomendasi untuk pembaca
i = 0
y= 0
print("***TESTCASE 1*** ")
print("Karena anda menyukai webtoon ", title , " mungkin kamu juga menyukai ini:")
for item in sorted_scores:
  comic_name = df[df.id == item[0]]['Name'].values[0]
  print(i+1,comic_name)
  i = i + 1
  if i > 9:
    break

print("\n")
print("***TESTCASE 2***")
#Tampilkan hasil rekomendasi untuk pembaca
print("Karena anda menyukai webtoon ", title2 , " mungkin kamu juga menyukai ini:")
for item in sorted_scores2:
  comic_name = df[df.id == item[0]]['Name'].values[0]
  print(y+1,comic_name)
  y = y + 1
  if y > 9:
    break

***TESTCASE 1*** 
Karena anda menyukai webtoon  About Death  mungkin kamu juga menyukai ini:
1 Death's Game
2 ShootAround
3 The Horizon
4 Gourmet Hound
5 Annarasumanara
6 Ghost Theater
7 Dark Mortal
8 Your Letter
9 Days of Hana
10 The Golden Spoon


***TESTCASE 2***
Karena anda menyukai webtoon  Delusion  mungkin kamu juga menyukai ini:
1 Dear X
2 Ctrl+Z
3 Rotten
4 Nightmare Factory
5 FLOWAR
6 Grasp
7 Shriek
8 Chiller
9 Epilogue
10 Bite Me
